In [38]:
import requests
import pandas as pd
import ast
from rapidfuzz import process, fuzz

platforms_df
images1

In [39]:
consoles_list=['Game Boy','Super Nintendo Entertainment System','Nintendo 64','PlayStation 5','Nintendo Switch 2','Xbox','PlayStation 2','Game Boy Color',
              'Nintendo Switch','Xbox Series X|S','Next Generation Xbox','Game Boy Advance','GameCube','Nintendo DS','Xbox 360','PlayStation 3','Wii','Nintendo 3DS', 
              'PlayStation 4'
             ]

In [40]:
game_reviews=pd.read_csv('../data/reviews_and_sales.csv')

In [42]:
games_df=pd.read_csv('../data/games_df.csv')

game_names=games_df['game_name'].unique()

game_names=pd.DataFrame(game_names)
game_names_list=game_names[0].tolist()

C:\Users\asher\AppData\Local\Temp\ipykernel_70800\3119781666.py:1: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df=pd.read_csv('../data/games_df.csv')


In [18]:
len(game_reviews['Year_of_Release'].unique())

40

In [26]:
game_reviews_real=game_reviews['Name'].loc[game_reviews['Name'].isin(game_names)]

In [32]:
game_names.dtypes

0    object
dtype: object

In [48]:
# Create a list of names from reviews_and_sales to compare against
review_names = game_reviews['Name'].astype(str).tolist()

# For each game_name, find the best match in review_names
matches = games_df['game_name'].astype(str).apply(
    lambda x: process.extractOne(x, review_names, scorer=fuzz.token_sort_ratio)
)

In [55]:
games_df['game_name'] = games_df['game_name'].astype(str)
game_reviews['Name'] = game_reviews['Name'].astype(str)

# Compute similarity matrix in parallel
scores = process.cdist(
    game_reviews['Name'].tolist(),
    games_df['game_name'].tolist(),
    scorer=fuzz.token_sort_ratio,
    workers=-1  # use all CPU cores
)

# Get best matches and their scores
best_match_idx = np.argmax(scores, axis=1)
best_match_score = scores[np.arange(scores.shape[0]), best_match_idx]
best_matches = [games_df['game_name'].iloc[i] for i in best_match_idx]

# Add results back to reviews_and_sales
game_reviews['matched_name'] = best_matches
game_reviews['score'] = best_match_score

# Replace Name with matched_name if score >= threshold
threshold = 85
game_reviews.loc[game_reviews['score'] >= threshold, 'Name'] = \
   game_reviews.loc[game_reviews['score'] >= threshold, 'matched_name']

# Merge on aligned names
merged_df = pd.merge(
    games_df,
    game_reviews,
    left_on='game_name',
    right_on='Name',
    how='inner'
)

# merged_df now contains both datasets combined
print(merged_df.head())


   Unnamed: 0  expected_release_day  expected_release_month  \
0           5                     0                       0   
1           7                     0                       0   
2          10                     0                       0   
3          13                     0                       0   
4          14                     0                       0   

   expected_release_quarter  expected_release_year  game_id  \
0                         0                   1996      125   
1                         0                      0      243   
2                         0                      0      397   
3                         0                   1994      697   
4                         0                      0      810   

                              game_name original_release_date  Game Boy  \
0                          College Slam                   NaN  Game Boy   
1  Street Fighter II: The World Warrior            1991-02-06  Game Boy   
2       James Pon

In [60]:
merged_df.to_csv('first_look_games.csv')

In [58]:
merged_df.dtypes

Unnamed: 0                    int64
expected_release_day          int64
expected_release_month        int64
expected_release_quarter      int64
expected_release_year         int64
                             ...   
User_Count                  float64
Developer                    object
Rating                       object
matched_name                 object
score                       float32
Length: 62, dtype: object

In [52]:
game_reviews

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
